# Erzeugen tflite und c-code

Zur Verwendung für die Programmierung im nächsten Kapitel wird ein File im tflite-Format oder ein c-Array zur Integration in den Programmcode benötigt. Dieses wird hier aus dem gespeicherten H5-Format erzeugt.

0. Vorbereitung (Laden der Bibliotheken und Einstellungen)
1. Laden des neuronalen Netzes
2. Umwandlung in tflite-Format
3. Speichern als tflite-File
4. Speichern als unsigned byte Array in c-File

### 0) Vorbereitung

##### Modelname
ACHTUNG: hier muss der korrekte Modelname definiert werden, wie er auch im letzten Skript verwendet wurde.

##### Bibliotheken
Laden der notwendigen Bibliotheken.

In [ ]:
###############################################################################

ModelNameAndVersion = "dig-01"

###############################################################################

from PIL import Image 
import numpy as np

import tensorflow as tf

### 1) Laden des neuronalen Netzes

Mittels der Funktion "load_model" wird das Model geladen.

In [ ]:
model = tf.keras.models.load_model('saved_model/' + ModelNameAndVersion)

# model.summary()

### 2) Umwandlung in tflite-Format

Mittels der TFliteConverter-Funktion wird das Model in das kompakte tflite-Format umgewandelt und anschließend als Binaryfile mit der Endung ".tflite" gespeichert.

Dieses Format kann im dritten Teil dann direkt in das entsprechende c-Modul geladen werden.

In [ ]:
## tflite-Format
converter    = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()


### 3) Speichern im Dateiformat

Das tflite-Model kann sehr einfach als Binardatei in ein File geschrieben werden. Als Endung wird hier "tflite" verwendet.

In [ ]:
open(ModelNameAndVersion + ".tfl", "wb").write(tflite_model)

### 4) Speichern als unsigned byte Array in c-File

Wenn man im späteren Progammiercode das neuronale Netz direkt im Quelltext speichern will, so ist dies über ein byte-Array möglich, welches das tflite-File enthält. Mittels einer Hilfsfunktion wird dieses hier zunächst als String erzeugt und dann in ein c-headerfile gespeichert.

Zusätzlich zu dem Array selbst noch die Größe in einer Variable gespeichert. Dieses h-File kann dann direkt in den Quellcode eingebunden werden.

In [ ]:
def bytes_to_c_arr(data):
    return [format(b, '#04x') for b in data]

c_code = "unsigned char tflite_model[] = \n"
c_code = c_code + "{{{}}};".format(", ".join(bytes_to_c_arr(tflite_model)))
c_code = c_code + "\n\nunsigned int tflite_model_len = "+ str(len(tflite_model)) + ";"
print(c_code)

open(ModelNameAndVersion + ".h", "w").write(c_code);